In [1]:
from bs4 import BeautifulSoup as BS
import re
from time import sleep
import requests
from yandexfreetranslate import YandexFreeTranslate
from multiprocessing.dummy import Pool as ThreadPool
from tqdm.notebook import tqdm
import pandas as pd
from pathlib import Path
from ebooklib import epub
from random import randint

In [2]:
folder = Path('books')
name = 'shadow-slave'
name = 'pestilence-rise-of-the-pure-undead'

In [3]:
url = 'https://novelbjn.novelupdates.net/ajax/chapter-option'

params = {
    'novelId': name,
}

res_links = requests.get(url=url, params=params, timeout=20)
page = BS(res_links.text, 'html.parser')
links = list(map(lambda x: x.get('value'), page.find('body').find_all('option')))


In [4]:
links

['https://novelbjn.novelupdates.net/book/pestilence-rise-of-the-pure-undead/chapter-1-witnessing-a-miracle',
 'https://novelbjn.novelupdates.net/book/pestilence-rise-of-the-pure-undead/chapter-2-no-time-for-you',
 'https://novelbjn.novelupdates.net/book/pestilence-rise-of-the-pure-undead/chapter-3-meaningless-floating-screen',
 'https://novelbjn.novelupdates.net/book/pestilence-rise-of-the-pure-undead/chapter-4-kin',
 'https://novelbjn.novelupdates.net/book/pestilence-rise-of-the-pure-undead/chapter-5-unacceptable',
 'https://novelbjn.novelupdates.net/book/pestilence-rise-of-the-pure-undead/chapter-6-still',
 'https://novelbjn.novelupdates.net/book/pestilence-rise-of-the-pure-undead/chapter-7-lying-in-ambush',
 'https://novelbjn.novelupdates.net/book/pestilence-rise-of-the-pure-undead/chapter-8-something-in-the-woods',
 'https://novelbjn.novelupdates.net/book/pestilence-rise-of-the-pure-undead/chapter-9-intrusion',
 'https://novelbjn.novelupdates.net/book/pestilence-rise-of-the-pure-un

In [5]:
list(enumerate(links))[62]

(62,
 'https://novelbjn.novelupdates.net/book/pestilence-rise-of-the-pure-undead/chapter-63-double')

In [6]:
links = list(map(lambda x: re.sub(r'(.*:\/\/.*?\/)', 'https://novelbjn.novelmagic.org/', x), links))

In [7]:
book = []

In [8]:
start = 0
start_book = len(links) - len(links[start:])

In [9]:
for url in tqdm(links[start:]):
    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36',
    }

    for _ in range(5):
        try:
            res_page = requests.get(url, headers=headers, timeout=20)
        except requests.exceptions.Timeout:
            continue
        except requests.exceptions.ConnectionError:
            continue
        else:
            break
    else:
        raise(Exception('stop'))

    page = BS(res_page.text, 'html.parser').find('div', id='chr-content')
    

    
    paragraphs = BS(res_page.text, 'html.parser').find('div', id='chr-content').get_text(separator='\n', strip=True).split('\n')
    book.append(paragraphs)

  0%|          | 0/164 [00:00<?, ?it/s]

In [10]:
len(book)

164

In [11]:
book = pd.Series(book).drop_duplicates().to_list()

In [12]:
def _worker(pool_list: tuple[str, tqdm]) -> str:
    queue, pbar = pool_list
    if not queue.strip():
        pbar.update(1)
        return ''
    yt = YandexFreeTranslate(api = "ios")
    
    for _ in range(10):
        try:
            text = yt.translate("en", "ru", queue)
        except Exception:
            sleep(0.5)
            continue
        else:
            break
    else:
        text = queue
        pbar.write(str(queue))
    pbar.update(1)
    return text
    

def run_translate(lst: list[str], pbar: tqdm)-> list[str]:
    pbar.reset(len(lst))
    pool_list = list(zip(lst, [pbar]*len(lst)))
    with ThreadPool(8) as pool:
        workreturn = pool.map(_worker, pool_list)
    return workreturn

In [13]:
new_book = []

In [14]:
pbar1 = tqdm(desc='page')
pbar2 = tqdm(desc='row')
pbar1.reset(len(links[start:]))
for index, page in list(enumerate(book, start=start_book)):
    try:
        new_page = run_translate(page, pbar2)
        new_book.append(new_page)
    except Exception:
        print(index)
        break
    pbar1.update()
pbar1.close()
pbar2.close()
    

page: 0it [00:00, ?it/s]

row: 0it [00:00, ?it/s]

In [15]:
# Функция для преобразования страницы в HTML
def page_to_html(page, title):
    content_lines = page[:]
    # Формирование HTML-кода
    html = f"<h1>{title}</h1>"
    for line in content_lines:
        html += f"<p>{line}</p>"
    return html


title = name

# Создание новой книги
epub_book = epub.EpubBook()

# Установка метаданных
epub_book.set_identifier(f"id{randint(100000,1000000)}")
epub_book.set_title(title)
epub_book.set_language("ru")

# Автор
epub_book.add_author("None")
# epub_book.spine.append('nav')
# Обработка каждой страницы
for i, page in enumerate(new_book, start=start_book):
    # Создание главы
    chapter_title = f'Глава {i+1:04d}'
    chapter_content = page_to_html(page, chapter_title)
    
    chapter = epub.EpubHtml(title=chapter_title, file_name=f'chap_{i+1:04d}.xhtml', lang='ru')
    chapter.content = chapter_content
    
    epub_book.add_item(chapter)
    
    # Добавление в оглавление
    epub_book.toc.append(chapter)
    # Упаковка всех глав в книгу
    epub_book.spine.append(chapter)


# Добавление оглавления
epub_book.add_item(epub.EpubNcx())
epub_book.add_item(epub.EpubNav())

# Сохранение книги
epub.write_epub(folder.joinpath(f'{name}.epub'), epub_book)
